In [13]:
#imports
import numpy as np
import numpy.random as rd
import numpy.linalg as linalg
import math
from scipy.optimize import linprog
import matplotlib.pyplot as plt
from sklearn import linear_model

In [4]:
def sample(prob): # BS can be done
    r = rd.random();i = 0
    while r > prob[i]:
        r-=prob[i]
        i+=1
    return i

def bernoulli(prob):
    if rd.random() > prob:
        return 1.0
    return 0.0

def row_normalize(arr):
    arr = np.array(arr,float)
    for i in range(np.shape(arr)[0]):
        arr[i] = arr[i]/np.sum(arr[i])
    return arr

def dot(x,y):
    return np.sum(np.multiply(x,y))

In [7]:
# test case 1
en = 6
ch = 8
P = np.array([.3,.1,.2,.4,.3,.1,.3,.3])/2
E = np.array([[.4,.6,.4,.3,.1,.35,.3,.4],[.3,.4,.4,.3,.3,.35,.25,.4],[.3,.2,.1,.4,.3,.4,.4,.3],[.4,.3,.3,.5,.6,.4,.55,.35],[.2,.4,.1,.3,.4,.4,.3,.1],[.4,.6,.4,.3,.6,.4,.3,.1]])/2
R = np.array([4.,3.5,4.,8.,2.,3.])
tol = .2
print(P)
print(E)
print(R)
print(tol)

[0.15 0.05 0.1  0.2  0.15 0.05 0.15 0.15]
[[0.2   0.3   0.2   0.15  0.05  0.175 0.15  0.2  ]
 [0.15  0.2   0.2   0.15  0.15  0.175 0.125 0.2  ]
 [0.15  0.1   0.05  0.2   0.15  0.2   0.2   0.15 ]
 [0.2   0.15  0.15  0.25  0.3   0.2   0.275 0.175]
 [0.1   0.2   0.05  0.15  0.2   0.2   0.15  0.05 ]
 [0.2   0.3   0.2   0.15  0.3   0.2   0.15  0.05 ]]
[4.  3.5 4.  8.  2.  3. ]
0.2


In [8]:
# finding optimal x
ub = np.sum(E*P,axis=1)
ltemp = linprog(c = -1*R, A_ub = [ub], b_ub = [tol],A_eq = np.ones((1,en)),b_eq = [1.0],method='simplex')
print(min(ub))
print(dot(ltemp.x,R), np.sum(ltemp.x),ltemp.con, ltemp.success, dot(ltemp.x,ub))
org_x = ltemp.x
print(ltemp.x)

0.13
6.518518518518519 1.0 [0.] True 0.2
[0.         0.         0.37037037 0.62962963 0.         0.        ]


In [14]:
# finding safe x0
ub = E.transpose();tol_arr = [tol]*ch
ltemp = linprog(c=np.zeros(en),A_ub = ub,b_ub=tol_arr,A_eq = np.ones((1,en)),b_eq = [1.0],method='revised simplex',options={'tol': 1.0e-3})
x0 = np.transpose(row_normalize([ltemp.x+(0.0)])) # noise to safe to avoid singular matrices, 1.0e-2 is working
norm_x0 = linalg.norm(x0)
e0 = x0/norm_x0 # safe vector
e0_trans = e0.transpose()
en0 = sample(x0.transpose()[0])
b0 = bernoulli(E[en0][sample(P)])
yt = (E[en0]*(1-b0))+((1-E[en0])*b0)
yt = row_normalize([np.exp(yt)]).transpose()
c0 = dot(x0,np.matmul(E,yt)) # safe cost
print(x0.flatten(),c0)

[0.5 0.  0.5 0.  0.  0. ] 0.1635037574449184


In [66]:
# parameters
T = 5000
S = 20
T_start = 30
# init
c = np.zeros(T)
y = np.zeros((T,en))
x = np.zeros((T,en))
x[0] = x0.flatten()
c[0] = c0
reg = linear_model.LinearRegression(fit_intercept=False)
avg_rate = []

In [68]:
for t in range(1,T+1):
    b = np.zeros(S)
    for s in range(S):
        b[s] = bernoulli(E[sample(x[t-1])][sample(P)])
    c[t] = np.mean(b)
    if t <= T_start:
        var = x0.copy().flatten()
        var[rd.randint(en)] += 0.1
        x[t] = var/np.sum(var)
        continue
    reg.fit(x[:t],c[:t])
    y[t-1] = reg.coef_/
    try:
        ltemp = linprog(c = -1*R, A_ub = [y[t-1]], b_ub = [tol],A_eq = np.ones((1,en)),b_eq = [1.0],method='revised simplex',options={'tol': 1.0e-2})
    except:
        print('skipped')
        x[t] = x[t-1]
        continue
    x[t] = ltemp.x
    avg_rate.append(dot(x[t],R))
    print(t,avg_rate[-1],dot(x[t],y[t-1]),ltemp.success)

31 6.376665644634411 0.2 True
32 8.0 0.13124515583179713 True
33 8.0 0.13660316521880506 True
34 8.0 0.1253724096525015 True
35 8.0 0.12924544138385044 True
36 8.0 0.13122814304839572 True
37 8.0 0.12758071082875586 True
38 8.0 0.12715308290669702 True
39 8.0 0.1259595824847711 True
40 8.0 0.1250556910287415 True
41 8.0 0.12224904047695204 True
42 8.0 0.12314728908976288 True
43 8.0 0.1215816899063969 True
44 8.0 0.1234437151461374 True
45 8.0 0.12452859489764295 True
46 8.0 0.12545666301805203 True
47 8.0 0.1254227355022639 True
48 8.0 0.12460553268985332 True
49 8.0 0.12388162614647388 True
50 8.0 0.12217606589097661 True
51 8.0 0.12232125255161853 True
52 8.0 0.12149479950618927 True
53 8.0 0.12104952706796399 True
54 8.0 0.12093533152568181 True
55 8.0 0.12110967351945301 True
56 8.0 0.12126947725155227 True
57 8.0 0.12116024178732017 True
58 8.0 0.12081256325623682 True
59 8.0 0.12144068874822735 True
60 8.0 0.12179448092197703 True
61 8.0 0.121461565955785 True
62 8.0 0.120722341

IndexError: index 5000 is out of bounds for axis 0 with size 5000

In [70]:
y

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       ...,
       [0.10074816, 0.2188878 , 0.14182747, 0.12174375, 0.2127164 ,
        0.20407643],
       [0.10074562, 0.21888744, 0.14182907, 0.12174569, 0.21271606,
        0.20407612],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])